## Cheatsheat para mapas con leaflet

In [ ]:
library(sf)
library(leaflet)

Para desplegar mapas

In [ ]:
mp1 <- leaflet(mxpolig) %>%
    addTiles() %>%
    setView(lng = -99.2, lat = 19.34, zoom = 7) # En setview establecemos donde queremos la vista inicial

mp1

Para añadir informacion adicional usando un dataframe auxiliar

In [ ]:
mp2 <- leaflet(mxpolig) %>%
    addTiles() %>%
    addMarkers(data = df1) %>%
    setView(lng = -99.2, lat = 19.34, zoom = 4.4)
mp2

Si queremos añadir marcadores para mostrar informacion adicional utilizamos el metodo `addMarkers()` junto con `poup=paste0()`

In [ ]:
mp2 <- mp2 %>%
    addMarkers(data = df1, popup = paste0( # para añadir markers primero indicas de donde los vas a sacar
        "<strong>Nombre: </strong>", df1$NOM_ENT, "<br>", # Posteriormente indicas la columna de ese df
        "<strong>Población: </strong>", df1$POB, "<br>"
    ))
mp2

Si quieres marcar los clusters se usa `clusterOptions= markerClusterOptions()`

In [ ]:
mp3 <- leaflet(mxpolig) %>%
    addTiles() %>%
    addMarkers(
        data = df1, popup = paste0(
            "<strong>Nombre: </strong>", df1$NOM_ENT, "<br>",
            "<strong>Poblaci?n: </strong>", df1$POB, "<br>"
        ),
        clusterOptions = markerClusterOptions()
    ) # Esta opcion es por indicar los clusters
mp3


Para hacer mapas de coropletas:

**NOTA:**
Usamos el metodo `addPolygons()` ahí indicamos que columna tomar como base con `fillColor= ~colores(df$col)`

In [ ]:
alPob <- colorNumeric("Blues", domain = df1$POB) # Elegimos el tipo de paleta que utilizaremos y de donde sacara los datos

mp5 <- leaflet(mxpolig) %>%
    addTiles() %>%
    addMarkers(
        data = df1, popup = paste0(
            "<strong>Nombre: </strong>", df1$NOM_ENT, "<br>",
            "<strong>Población: </strong>", df1$POB, "<br>"
        ),
        clusterOptions = markerClusterOptions()
    ) %>%
    addPolygons(
        stroke = F, # En addPolygons indicaps como se van a colorear los poligonos
        smoothFactor = 0.2,
        opacity = 1.0,
        fillOpacity = 0.5,
        fillColor = ~ palPob(df1$POB),
        highlightOptions = highlightOptions(
            color = "white",
            weight = 2,
            bringToFront = T
        ),
        label = df1$NOM_ENT,
        labelOptions = labelOptions(direction = "auto")
    ) %>%
    addLegend(
        position = "bottomleft", pal = palPob, values = ~ df1$POB,
        title = "Población"
    )


mp5


## **IMPORTANTE**
Si los datos son bastante grandes para la escala conviene primero definir los limites del intervalo y con esos limites establecer el mapa. 

Para eso utilizas el siguiente metodo:
1. `bins <- c(0, 20, 45, 79, 150, 390, 800, 7000) ` para indicar los limites 
2. `palDens <- colorBin("Greens", domain = df1$DENS, bins=bins, pretty = F)` Escala de color junto con los limites de corte

In [ ]:
mp6 <- leaflet(mxpolig) %>%
    addTiles() %>%
    addMarkers(
        data = df1, popup = paste0(
            "<strong>Nombre: </strong>", df1$NOM_ENT, "<br>",
            "<strong>Población: </strong>", df1$POB, "<br>"
        ),
        clusterOptions = markerClusterOptions()
    ) %>%
    addPolygons(
        stroke = F, # En addPolygons indicaps como se van a colorear los poligonos
        smoothFactor = 0.2,
        opacity = 1.0,
        fillOpacity = 0.5,
        fillColor = ~ palDens(df1$DENS),
        highlightOptions = highlightOptions(
            color = "white",
            weight = 2,
            bringToFront = T
        ),
        label = df1$NOM_ENT,
        labelOptions = labelOptions(direction = "auto")
    ) %>%
    addLegend(
        position = "bottomleft", pal = palDens, values = ~ df1$DENS,
        title = "Población"
    )


mp6

## **Para unir varios Mapas**

1. Se establece `addPolygons()`con los parametros a colorear del mapa
2. Establecemos `addLegend()`para mostrar la leyenda de colores
3. Vuelves a repetir para el segundo mapa
4. Establecemos los marcadores `addMarkers()` (opcional)
5. Configuramos los botones con `addLayersControl()` para el nombre de cada boton usamos el parametro `overlayGroups()`

In [ ]:
mp10 <- leaflet(mxpolig) %>%
    addTiles() %>%
    addPolygons(
        stroke = F, smoothFactor = 0.2, opacity = 1.0,
        fillOpacity = 0.5, fillColor = ~ palPob(df1$POB),
        highlightOptions = highlightOptions(
            color = "white",
            weight = 2,
            bringToFront = T
        ),
        label = df1$NOM_ENT, labelOptions = labelOptions(direction = "auto"),
        group = "Por Población"
    ) %>%
    addLegend(
        position = "bottomleft", pal = palPob, values = ~ df1$POB,
        title = "Población", group = "Por Población"
    ) %>%
    addPolygons(
        stroke = F, smoothFactor = 0.2, opacity = 1.0,
        fillOpacity = 0.5, fillColor = ~ palDens(df1$DENS),
        highlightOptions = highlightOptions(
            color = "white",
            weight = 2,
            bringToFront = T
        ),
        label = df1$NOM_ENT, labelOptions = labelOptions(direction = "auto"),
        group = "Por Densidad"
    ) %>%
    addLegend(
        position = "bottomleft", pal = palDens, values = ~ df1$DENS,
        labFormat = labelFormat(digits = 0), group = "Por Densidad",
        title = "Densidad"
    ) %>%
    addMarkers(
        data = df1, popup = paste0(
            "<strong>Nombre: </strong>", df1$NOM_ENT, "<br>",
            "<strong>Población: </strong>", df1$POB, "<br>",
            "<strong>Densidad Poblacional: </strong>", df1$DENS, "<br>"
        ),
        clusterOptions = markerClusterOptions()
    ) %>%
    addProviderTiles(providers$CartoDB.Positron, group = "Mapa base 2") %>%
    addProviderTiles(providers$Esri.NatGeoWorldMap, group = "Mapa base 3") %>%
    addLayersControl(
        baseGroups = c("Mapa base 1", "Mapa base 2", "Mapa base 3"),
        overlayGroups = c("Por Población", "Por Densidad"),
        options = layersControlOptions(collapsed = F)
    )

mp10